
#PLAN OF ACTION AND PROGRESS MADE#

###1.Acquire lat long values from farmers via website###

 -*Automation of input accesing through API key is a future scope.*


###2.Create a rectanglar polygon of the given coordinates and access the google map image as KML file###


-*This step has been succesfully implemented and the required KML file has been uploaded in the google drive.*


###3.Upload the KML file into google earth engine assets so as to procure the sentinel satellite images of the area of interest###


-*This step has also been succesfully implemented  and the corresponding geotif image was imported into the google drive.*


###4.The  geotif image procured should be further processed through a software called SNAP(Sentinel Application Platform) to attain the 4 band imagery###

-*This step has not yet been automated and so is included in the scope.*



###*AUTHENTICATE TO GOOGLE EARTH ENGINE###







In [ ]:

!earthengine authenticate 


In [ ]:
!pip install earthengine-api #earth-engine Python API

###*AUTHENTICATE TO GOOGLE DRIVE###

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###* AUTHENTICATE TO GOOGLE CLOUD###

In [10]:
from google.colab import auth
auth.authenticate_user()

###* SETTING UP##




In [ ]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

In [12]:
import ee

ee.Initialize()


In [14]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

###* ACCESSING THE SATELLITE IMAGE OF THE INPUT COORDINATES###



In [61]:
# Load a landsat image and select three bands.

sentinel = ee.Image('COPERNICUS/S2/20151128T002653_20151128T102149_T56MNN')\
            .select(['B5','B4', 'B3', 'B2'])

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([79.272713,28.354472,80.01236,29.4849])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center,{'sentinel':sentinel.getMapId()},zoom_start=7)

###*EXPORTING THE IMAGE TO GOOGLE DRIVE###

In [62]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': landsat,
    'description': 'imageToDriveExample',
    'folder':'Example_folder',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [63]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).
Polling for task (id: 6IVGTSIDY463S33UI65THCI3).


In [66]:
# Get band 4 from the Landsat image, copy it.
band4 = sentinel.select('B4').rename('b4_mean')\
               .addBands(sentinel.select('B4').rename('b4_sample'))\
               .addBands(sentinel.select('B4').rename('b4_max'))\

# Export the image to an Earth Engine asset.
task = ee.batch.Export.image.toAsset(**{
  'image': band4,
  'description': 'imageToAssetExample',
  'assetId': 'users/reshmasanthoshs/export',
  'scale': 100,
  'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [68]:
# Make a collection of points.
features = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(79.272713,28.354472), {'name': 'Gularia'}),
  ee.Feature(ee.Geometry.Point(77.3910,28.5355), {'name': 'Noida'}),
  ee.Feature(ee.Geometry.Point(77.3178,28.4089 ), {'name': 'Faridabad'})
])
center = features.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':features.getInfo()},zoom_start=2)

In [69]:
# Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(**{
  'collection': features,
  'description':'vectorsToDriveExample',
  'fileFormat': 'SHP'
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [49]:
# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toAsset(**{
    'collection': features,
    'description':'exportToTableAssetExample',
    'assetId': 'users/reshmasanthosh/ibm'
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)